In [ ]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv('표준코드_산지코드.csv', encoding='cp949')

# 숫자 비교를 위해 'plor_cd' 정수형으로 변환
df['plor_cd'] = pd.to_numeric(df['plor_cd'], errors='coerce')

# 제외할 지역명 접두어
excluded_prefixes = ['서울', '대전', '세종', '인천', '광주', '부산', '울산', '대구']

# 제외 조건: plor_cd < 800000 AND plor_nm이 제외 접두어로 시작하지 않는 것
mask = (df['plor_cd'] < 800000) & (~df['plor_nm'].str.startswith(tuple(excluded_prefixes)))

# 필터링
filtered_df = df[mask]

# 결과 저장
filtered_df.to_csv('filtered_산지코드.csv', index=False, encoding='cp949')

In [ ]:
import pandas as pd

# CSV 파일 경로
file_path = 'filtered_산지코드.csv'

# CSV 파일 읽기 (한글 인코딩 고려)
df = pd.read_csv(file_path, encoding='cp949')

# 축약형 -> 전체 행정구역 이름 매핑
region_map = {
    '강원': '강원도',
    '경기': '경기도',
    '경남': '경상남도',
    '경북': '경상북도',
    '광주': '광주광역시',
    '대구': '대구광역시',
    '대전': '대전광역시',
    '부산': '부산광역시',
    '서울': '서울특별시',
    '세종': '세종특별자치시',
    '울산': '울산광역시',
    '인천': '인천광역시',
    '전남': '전라남도',
    '전북': '전라북도',
    '제주': '제주특별자치도',
    '충남': '충청남도',
    '충북': '충청북도'
}

# 안전한 교체 함수
def replace_region_name_safe(plor_nm):
    for short, full in region_map.items():
        if plor_nm.startswith(full):  # 이미 전체 이름이면 그대로
            return plor_nm
        elif plor_nm.startswith(short):  # 축약형이면 교체
            return full + plor_nm[len(short):]
    return plor_nm  # 매칭 안 되면 그대로

# 적용
df['plor_nm'] = df['plor_nm'].apply(replace_region_name_safe)

# 결과 저장 (선택사항)
df.to_csv('updated_산지코드.csv', index=False, encoding='cp949')

In [ ]:
import pandas as pd

# CSV 파일 경로
file_path = 'updated_산지코드.csv'

# CSV 파일 읽기 (한글 인코딩 고려)
df = pd.read_csv(file_path, encoding='cp949')

# 도 + 시/군까지만 추출 (예: '강원도 춘천시')
df['region_city'] = df['plor_nm'].str.extract(r'^([^\s]+ [^\s]+)')

# plor_cd를 정수형으로 변환
df['plor_cd'] = df['plor_cd'].astype(int)

# 연속된 코드 묶기
def condense_ranges(codes):
    codes = sorted(set(codes))
    ranges = []
    start = prev = codes[0]

    for code in codes[1:]:
        if code == prev + 1:
            prev = code
        else:
            if start == prev:
                ranges.append(f"{start}")
            else:
                ranges.append(f"{start}~{prev}")
            start = prev = code

    if start == prev:
        ranges.append(f"{start}")
    else:
        ranges.append(f"{start}~{prev}")

    return ','.join(ranges)

# 지역별 plor_cd 묶기
grouped = df.groupby('region_city')['plor_cd'].apply(condense_ranges).reset_index()
grouped.columns = ['region_city', 'plor_cd_range']

# 결과 확인 또는 저장
print(grouped.head())

# 저장 (선택)
grouped.to_csv('지역별_plor_cd범위.csv', index=False, encoding='cp949')

In [ ]:
import pandas as pd
import re

# CSV 불러오기
df_standard = pd.read_csv("표준코드_산지코드.csv", encoding="cp949")
df_updated = pd.read_csv("updated_산지코드.csv", encoding="cp949")

# 숫자형 처리
df_standard["plor_cd"] = pd.to_numeric(df_standard["plor_cd"], errors="coerce")
df_updated["plor_cd"] = pd.to_numeric(df_updated["plor_cd"], errors="coerce")
df_standard = df_standard.dropna(subset=["plor_cd"])
df_updated = df_updated.dropna(subset=["plor_cd"])
df_standard["plor_cd"] = df_standard["plor_cd"].astype(int)
df_updated["plor_cd"] = df_updated["plor_cd"].astype(int)

# 필터링
df_filtered = df_standard[
    (~df_standard["plor_cd"].isin(df_updated["plor_cd"])) &
    (df_standard["plor_cd"] < 800000)
].copy()

# 행정명칭 매핑
city_map = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
    "세종": "세종특별자치시"
}

# 정제 적용
for short, full in city_map.items():
    # 세종은 특별 처리: 세종, 세종시로 시작하지만 세종특별자치시 아닌 경우만 치환
    if short == "세종":
        df_filtered["plor_nm"] = df_filtered["plor_nm"].apply(
            lambda x: re.sub(r"^세종(시)?(?!특별자치시)", full, x)
            if not x.startswith("세종특별자치시") else x
        )
    else:
        pattern = rf"^{short}(?!시|특별시|광역시|특별자치시)"
        df_filtered["plor_nm"] = df_filtered["plor_nm"].apply(
            lambda x: re.sub(pattern, full, x)
        )

    
# 결과 저장 (선택)
df_filtered.to_csv("표준코드_광역시_산지코드.csv", index=False, encoding="cp949")

In [ ]:
import pandas as pd

# CSV 파일 로드
df_base = pd.read_csv("표준코드_산지코드.csv", encoding="cp949")
df_over1 = pd.read_csv("표준코드_광역시_산지코드.csv", encoding="cp949")
df_over2 = pd.read_csv("updated_산지코드.csv", encoding="cp949")

# 모든 DataFrame에서 plor_cd 정리
for df in [df_base, df_over1, df_over2]:
    df["plor_cd"] = pd.to_numeric(df["plor_cd"], errors="coerce")
    df.dropna(subset=["plor_cd"], inplace=True)
    df["plor_cd"] = df["plor_cd"].astype(int)

# 덮어쓰기 대상 병합 + plor_cd 중복 제거 (나중 값 우선)
df_override = pd.concat([df_over1, df_over2], ignore_index=True)
df_override = df_override.drop_duplicates(subset="plor_cd", keep="last")

# plor_cd 기준으로 df_base 덮어쓰기
df_base.set_index("plor_cd", inplace=True)
df_override.set_index("plor_cd", inplace=True)

df_combined = df_override.combine_first(df_base).reset_index()


# 'plor_cd' 열을 숫자형으로 변환 (오류 시 NaN 처리), 이후 정수형(Int64)으로 변환
df['plor_cd'] = pd.to_numeric(df['plor_cd'], errors='coerce').astype('Int64')

# 결과 저장 (선택)
df_combined.to_csv("표준코드_덮어쓰기완료.csv", index=False, encoding="cp949")

In [ ]:
import pandas as pd
import re

# CSV 불러오기
df = pd.read_csv("표준코드_덮어쓰기완료.csv", encoding='cp949')

# 주소에서 시도명 또는 시도 + 시/군 추출
def extract_main_region(name):
    if pd.isna(name):
        return pd.NA
    
    # 특별시, 광역시, 특별자치시 → 시도명까지만 추출
    match_city_only = re.match(r'^(.*?(특별시|광역시|특별자치시))', name)
    if match_city_only:
        return match_city_only.group(1)
    
    # 도, 특별자치도 → 시도 + 시/군까지 추출
    match_province = re.match(r'^(.*?(도|특별자치도))\s+(\S+?(시|군))', name)
    if match_province:
        return f"{match_province.group(1)} {match_province.group(3)}"
    
    return name  # 매치되지 않으면 원본 반환

# 적용
df['plor_nm'] = df['plor_nm'].apply(extract_main_region)

# 저장
df.to_csv("표준코드_정리완료.csv", index=False, encoding='cp949')

In [ ]:
import pandas as pd
from tqdm import tqdm

# CSV 파일 로딩
df = pd.read_csv("표준코드_정리완료.csv", encoding='cp949')

df['plor_cd'] = pd.to_numeric(df['plor_cd'], errors='coerce')

# 정렬
df = df.sort_values('plor_cd').reset_index(drop=True)

# 1. 800000 미만 처리
df_lt = df[df['plor_cd'] < 800000].reset_index(drop=True)

range_rows = []
i = 0
with tqdm(total=len(df_lt)) as pbar:
    while i < len(df_lt):
        start_cd = df_lt.loc[i, 'plor_cd']
        plor_nm = df_lt.loc[i, 'plor_nm']
        j = i

        # 같은 plor_nm 연속 구간
        while j + 1 < len(df_lt) and df_lt.loc[j + 1, 'plor_nm'] == plor_nm:
            j += 1

        # 종료값 결정
        if j + 1 < len(df_lt):
            end_cd = df_lt.loc[j + 1, 'plor_cd'] - 1
        else:
            end_cd = 799999  # 마지막 구간은 강제 고정

        # 범위 문자열
        code_range = f"{start_cd}" if start_cd == end_cd else f"{start_cd}~{end_cd}"

        range_rows.append({
            '코드범위': code_range,
            'plor_nm': plor_nm
        })

        tqdm.write(f"처리됨: {plor_nm} ({code_range})")
        pbar.update(j - i + 1)
        i = j + 1

# 2. 800000 이상 원본 처리
df_gte = df[df['plor_cd'] >= 800000].copy()
df_gte['코드범위'] = df_gte['plor_cd'].astype(str)
df_gte['plor_nm'] = df_gte['plor_nm']
df_gte = df_gte[['코드범위', 'plor_nm']]

# 3. 병합 및 저장
result_df = pd.concat([pd.DataFrame(range_rows), df_gte], ignore_index=True)

# 저장
result_df.to_csv("표준코드_정리완료1.csv", index=False, encoding='cp949')

print("✅ 저장 완료: 표준코드_정리완료1.csv")

In [ ]:
import pandas as pd

# 파일 경로
file_path = "표준코드_정리완료.csv"

# CSV 파일 불러오기 (인코딩: cp949)
df = pd.read_csv(file_path, encoding='cp949')

# '코드범위' 컬럼명을 'plor_cd'로 변경
df = df.rename(columns={'코드범위': 'plor_cd'})

# 고유한 plor_nm 기준으로 new_cd, new_nm 생성
df_mapping = df.drop_duplicates(subset='plor_nm').reset_index(drop=True)
df_mapping['new_cd'] = range(1000, 1000 + len(df_mapping))
df_mapping['new_nm'] = df_mapping['plor_nm']

# 전체 원본 데이터에 매핑값 병합
df_full = df.merge(df_mapping[['plor_nm', 'new_cd', 'new_nm']], on='plor_nm', how='left')

# 결과 확인
print(df_full.head())

# 저장 (원할 경우)
df_full.to_csv("표준코드_new.csv", index=False, encoding='cp949')

In [ ]:
import pandas as pd

# 파일 경로
file_path = "산지코드_new.csv"

# CSV 파일 불러오기 (인코딩 cp949)
df = pd.read_csv(file_path, encoding='cp949')

# 컬럼명 변경
df_renamed = df.rename(columns={
    'plor_cd': '산지코드',
    'plor_nm': '산지이름',
    'new_cd': '직팜산지코드',
    'new_nm': '직팜산지이름'
})

# 결과 확인
print(df_renamed.head())

# 저장 (필요시)
df_renamed.to_csv("산지코드_직팜.csv", index=False, encoding='cp949')

In [ ]:
import pandas as pd

# 1. 파일 경로 설정
summary_path = "유통공사_배추_요약데이터_20250708.csv"
region_code_path = "산지코드_직팜.csv"

# 2. CSV 불러오기
summary_df = pd.read_csv(summary_path, encoding='cp949')
region_df = pd.read_csv(region_code_path, encoding='cp949')

# 3. 산지코드 범위 파싱 함수
def parse_code_range(code_str):
    if "~" in code_str:
        start, end = code_str.split("~")
    else:
        start = end = code_str
    return int(start), int(end)

# 4. 산지코드 범위 숫자 컬럼 생성
region_df[['start_code', 'end_code']] = region_df['산지코드'].apply(
    lambda x: pd.Series(parse_code_range(x))
)

# 5. 산지코드 → 직팜산지코드/이름 매핑 테이블 확장
expanded_rows = []
for _, row in region_df.iterrows():
    for code in range(row['start_code'], row['end_code'] + 1):
        expanded_rows.append({
            '산지코드': code,
            '직팜산지코드': row['직팜산지코드'],
            '직팜산지이름': row['직팜산지이름']
        })
expanded_map_df = pd.DataFrame(expanded_rows)

# 6. 요약 데이터 산지코드 정수형으로 변환
summary_df['산지코드'] = pd.to_numeric(summary_df['산지코드'], errors='coerce').astype('Int64')
summary_df = summary_df[summary_df['산지코드'].notna()].copy()
summary_df['산지코드'] = summary_df['산지코드'].astype(int)

# 7. 직팜 컬럼 제거 후 병합
summary_df = summary_df.drop(columns=['직팜산지코드', '직팜산지이름'], errors='ignore')
summary_df = pd.merge(summary_df, expanded_map_df, on='산지코드', how='left')

# 8. 산지이름이 누락된 경우 직팜산지이름으로 보완
summary_df['산지이름'] = summary_df['산지이름'].fillna(summary_df['직팜산지이름'])


# 9. 결과 저장
summary_df.to_csv("배추요약데이터_직팜정리.csv", index=False, encoding='cp949')